# Pset 3: Knights and Knaves

The objective in this pset is to write a program to solve logic puzzles using concepts of **Knowledge and Reasoining.**

NB: be sure to run all the code cells as you work your way through the notebook.


In [ ]:
# Install and import testing funtionality
%pip install -i https://test.pypi.org/simple/ fee232==0.1.5
from knowledge.knowledge import *

# Background
In 1978, logician Raymond Smullyan published “What is the name of this book?”, a book of logical puzzles. Among the puzzles in the book were a class of puzzles that Smullyan called “Knights and Knaves” puzzles.

In a Knights and Knaves puzzle, the following information is given: 
- Each character is either a knight or a knave. 
- A knight will always tell the truth: if a knight states a sentence, then that sentence is true. 
- Conversely, a knave will always lie: if a knave states a sentence, then that sentence is false.

The objective of the puzzle is, given a set of sentences spoken by each of the characters, determine, for each character, whether that character is a knight or a knave.

For example, consider a simple puzzle with just a single character named A. A says “I am both a knight and a knave.”

Logically, we might reason that if A were a knight, then that sentence would have to be true. But we know that the sentence cannot possibly be true, because A cannot be both a knight and a knave – we know that each character is either a knight or a knave, but not both. So, we could conclude, A must be a knave.

That puzzle was on the simpler side. With more characters and more sentences, the puzzles can get trickier! Your task in this problem is to determine how to represent these puzzles using propositional logic, such that an AI running a model-checking algorithm could solve these puzzles for us.


# Understanding

Take a look at the code cell below (_Logic_). _No need to understand everything in it_, but notice that the code cell defines several classes for different types of logical connectives. These classes can be composed within each other, so an expression like `And(Not(A)`, `Or(B, C))` represents the logical sentence stating that symbol A is not true, and that symbol B or symbol C is true (where “or” here refers to inclusive, not exclusive, or).

The code cell below also contains a function `model_check`. `model_check` takes a `knowledge base` and a `query`. The `knowledge base` is a single logical sentence: if multiple logical sentences are known, they can be joined together in an `And` expression. `model_check` recursively considers all possible models, and returns `True` if the `knowledge base` entails the `query`, and returns `False` otherwise.


## Logic

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL

# -*- coding: utf-8 -*-
import itertools


class Sentence():

    def evaluate(self, model):
        """Evaluates the logical sentence."""
        raise Exception("nothing to evaluate")

    def formula(self):
        """Returns string formula representing logical sentence."""
        return ""

    def symbols(self):
        """Returns a set of all symbols in the logical sentence."""
        return set()

    @classmethod
    def validate(cls, sentence):
        if not isinstance(sentence, Sentence):
            raise TypeError("must be a logical sentence")

    @classmethod
    def parenthesize(cls, s):
        """Parenthesizes an expression if not already parenthesized."""
        def balanced(s):
            """Checks if a string has balanced parentheses."""
            count = 0
            for c in s:
                if c == "(":
                    count += 1
                elif c == ")":
                    if count <= 0:
                        return False
                    count -= 1
            return count == 0
        if not len(s) or s.isalpha() or (
            s[0] == "(" and s[-1] == ")" and balanced(s[1:-1])
        ):
            return s
        else:
            # return f"({s})"
            return "({})".format(s)


class Symbol(Sentence):

    def __init__(self, name):
        self.name = name

    def __eq__(self, other):
        return isinstance(other, Symbol) and self.name == other.name

    def __hash__(self):
        return hash(("symbol", self.name))

    def __repr__(self):
        return self.name

    def evaluate(self, model):
        try:
            return bool(model[self.name])
        except KeyError:
            raise Exception("variable {} not in model".format(self.name))

    def formula(self):
        return self.name

    def symbols(self):
        return {self.name}


class Not(Sentence):
    def __init__(self, operand):
        Sentence.validate(operand)
        self.operand = operand

    def __eq__(self, other):
        return isinstance(other, Not) and self.operand == other.operand

    def __hash__(self):
        return hash(("not", hash(self.operand)))

    def __repr__(self):
        return "Not({})".format(self.operand)

    def evaluate(self, model):
        return not self.operand.evaluate(model)

    def formula(self):
        return "¬" + Sentence.parenthesize(self.operand.formula())

    def symbols(self):
        return self.operand.symbols()


class And(Sentence):
    def __init__(self, *conjuncts):
        for conjunct in conjuncts:
            Sentence.validate(conjunct)
        self.conjuncts = list(conjuncts)

    def __eq__(self, other):
        return isinstance(other, And) and self.conjuncts == other.conjuncts

    def __hash__(self):
        return hash(
            ("and", tuple(hash(conjunct) for conjunct in self.conjuncts))
        )

    def __repr__(self):
        conjunctions = ", ".join(
            [str(conjunct) for conjunct in self.conjuncts]
        )
        return "And({})".format(conjunctions)

    def add(self, conjunct):
        Sentence.validate(conjunct)
        self.conjuncts.append(conjunct)

    def evaluate(self, model):
        return all(conjunct.evaluate(model) for conjunct in self.conjuncts)

    def formula(self):
        if len(self.conjuncts) == 1:
            return self.conjuncts[0].formula()
        return " ∧ ".join([Sentence.parenthesize(conjunct.formula())
                           for conjunct in self.conjuncts])

    def symbols(self):
        return set.union(*[conjunct.symbols() for conjunct in self.conjuncts])


class Or(Sentence):
    def __init__(self, *disjuncts):
        for disjunct in disjuncts:
            Sentence.validate(disjunct)
        self.disjuncts = list(disjuncts)

    def __eq__(self, other):
        return isinstance(other, Or) and self.disjuncts == other.disjuncts

    def __hash__(self):
        return hash(
            ("or", tuple(hash(disjunct) for disjunct in self.disjuncts))
        )

    def __repr__(self):
        disjuncts = ", ".join([str(disjunct) for disjunct in self.disjuncts])
        return "Or({})".format(disjuncts)

    def evaluate(self, model):
        return any(disjunct.evaluate(model) for disjunct in self.disjuncts)

    def formula(self):
        if len(self.disjuncts) == 1:
            return self.disjuncts[0].formula()
        return " ∨  ".join([Sentence.parenthesize(disjunct.formula())
                            for disjunct in self.disjuncts])

    def symbols(self):
        return set.union(*[disjunct.symbols() for disjunct in self.disjuncts])


class Implication(Sentence):
    def __init__(self, antecedent, consequent):
        Sentence.validate(antecedent)
        Sentence.validate(consequent)
        self.antecedent = antecedent
        self.consequent = consequent

    def __eq__(self, other):
        return (isinstance(other, Implication)
                and self.antecedent == other.antecedent
                and self.consequent == other.consequent)

    def __hash__(self):
        return hash(("implies", hash(self.antecedent), hash(self.consequent)))

    def __repr__(self):
        return "Implication({}, {})".format(self.antecedent, self.consequent)

    def evaluate(self, model):
        return ((not self.antecedent.evaluate(model))
                or self.consequent.evaluate(model))

    def formula(self):
        antecedent = Sentence.parenthesize(self.antecedent.formula())
        consequent = Sentence.parenthesize(self.consequent.formula())
        return "{} => {}".format(antecedent, consequent)

    def symbols(self):
        return set.union(self.antecedent.symbols(), self.consequent.symbols())


class Biconditional(Sentence):
    def __init__(self, left, right):
        Sentence.validate(left)
        Sentence.validate(right)
        self.left = left
        self.right = right

    def __eq__(self, other):
        return (isinstance(other, Biconditional)
                and self.left == other.left
                and self.right == other.right)

    def __hash__(self):
        return hash(("biconditional", hash(self.left), hash(self.right)))

    def __repr__(self):
        return "Biconditional({}, {})".format(self.left, self.right)

    def evaluate(self, model):
        return ((self.left.evaluate(model)
                 and self.right.evaluate(model))
                or (not self.left.evaluate(model)
                    and not self.right.evaluate(model)))

    def formula(self):
        left = Sentence.parenthesize(str(self.left))
        right = Sentence.parenthesize(str(self.right))
        return "{} <=> {}".format(left, right)

    def symbols(self):
        return set.union(self.left.symbols(), self.right.symbols())


def model_check(knowledge, query):
    """Checks if knowledge base entails query."""

    def check_all(knowledge, query, symbols, model):
        """Checks if knowledge base entails query, given a particular model."""

        # If model has an assignment for each symbol
        if not symbols:

            # If knowledge base is true in model, then query must also be true
            if knowledge.evaluate(model):
                return query.evaluate(model)
            return True
        else:

            # Choose one of the remaining unused symbols
            remaining = symbols.copy()
            p = remaining.pop()

            # Create a model where the symbol is true
            model_true = model.copy()
            model_true[p] = True

            # Create a model where the symbol is false
            model_false = model.copy()
            model_false[p] = False

            # Ensure entailment holds in both models
            return (check_all(knowledge, query, remaining, model_true) and
                    check_all(knowledge, query, remaining, model_false))

    # Get all symbols in both knowledge and query
    symbols = set.union(knowledge.symbols(), query.symbols())

    # Check that knowledge entails query
    return check_all(knowledge, query, symbols, dict())

Ensure you have run the code cell above.

# Symbols


In the code cell below, six propositional symbols are defined. `AKnight`, for example, represents the sentence that “A is a knight,” while `AKnave` represents the sentence that “A is a knave.” Propositional symbols for characters B and C have also been defined.



In [ ]:
################## Symbols #########################
AKnight = Symbol("A is a Knight")
AKnave = Symbol("A is a Knave")

BKnight = Symbol("B is a Knight")
BKnave = Symbol("B is a Knave")

CKnight = Symbol("C is a Knight")
CKnave = Symbol("C is a Knave")


# Puzzle solver
In the `puzzle_solver()` function below, the symbols are specified in a list. The model checking function is then used to compute, given the knowledge for that puzzle, whether each character is a knight or a knave, printing out any conclusions that the model checking algorithm is able to make.
**You do not need to modfiy this function.**

In [ ]:
# You do not need to modify this code cell!

def puzzle_solver(puzzle_name, knowledge):
    symbols = [AKnight, AKnave, BKnight, BKnave, CKnight, CKnave]
    print(puzzle_name)

    if len(knowledge.conjuncts) == 0:
        print("    Not yet implemented.")
    else:
        for symbol in symbols:
            if model_check(knowledge, symbol):
                print(f"    {symbol}")

# Puzzles and Knowledge bases
In the cells below, there are six different knowledge bases, `knowledge0`, `knowledge1`, `knowledge2`, `knowledge3`, `knowledge4`,and `knowledge5`, which will contain the knowledge needed to deduce the solutions to the Puzzles 0, 1, 2, 3, 4 and 5, respectively. Notice that, for now, each of these knowledge bases is empty. That’s where you come in!


Add knowledge to knowledge bases `knowledge0`, `knowledge1`, `knowledge2`, `knowledge3`, `knowledge4`,and `knowledge5` to solve the respective puzzles.


In each of the puzzles, each character is either a knight or a knave. Every sentence spoken by a knight is true, and every sentence spoken by a knave is false.

Once you’ve completed the knowledge base for a problem, the `puzzle_sovler()` function is invoked to see the solution to the puzzle.

For each knowledge base, you’ll likely want to encode two different types of information:
1. Information about the structure of the problem itself (i.e., information given in the definition of a Knight and Knave puzzle), and 
2. Information about what the characters actually said. 


For the information of type (1):
- Think: how would you represent, using logical connectives, the fact that a character is **either** a knight or a knave?
- This  type of information will be present in all the knowledge bases.


For the information of type (2):
- Think: how would you represent, using logical connectives, the fact that a character **said** something?
- Consider what it means if a sentence is spoken by a character. Under what conditions is that sentence true? Under what conditions is that sentence false? How can you express that as a logical sentence?


There are multiple possible knowledge bases for each puzzle that will compute the correct result. You should attempt to choose a knowledge base that offers the most direct translation of the information in the puzzle, rather than performing logical reasoning on your own. You should also consider what the most concise representation of the information in the puzzle would be.

For instance, for Puzzle 0, setting `knowledge0 = AKnave` would result in correct output, since through our own reasoning we know A must be a knave. But doing so would **be against the spirit of this problem**: the goal is to have your AI do the reasoning for you.

## Puzzle 0
**Puzzle 0** is the puzzle from the Background. It contains a single character, A.
- A says “I am both a knight and a knave.”

Complete the code for the knowledge base and then run the cell to solve the puzzle.

In [ ]:
# Puzzle 0
# A says "I am both a knight and a knave."
A_says = And(AKnight, AKnave)

# Encode the knowledge in the knowledge base
knowledge0 = And(
    # A can be one and not both a knight and a knave
    Or(AKnight, AKnave),
    Not(And(AKnight, AKnave)),
    # test if what A says is true(knight) or false(knave)
    Implication(AKnight, A_says),
    Implication(AKnave, Not(A_says))
)

# Solve the puzzle
puzzle_solver('Puzzle 0', knowledge0)

After each puzzle, run the corresponding test to ensure that your code is correct. If the test runs without an error, then your knowledge base for the puzzle is correct.

In [ ]:
# Testing for knowledge0

# Ensure the symbols are defined
symbols = [AKnight, AKnave, BKnight, BKnave, CKnight, CKnave]

# Test knowledge0
test_0(model_check, knowledge0, symbols)

## Puzzle 1
**Puzzle 1** has two characters: A and B.
- A says “We are both knaves.”
- B says nothing.

In [ ]:
# Puzzle 1
# A says "We are both knaves."
A_says = And(AKnave, BKnave)
# B says nothing.
B_says = None

knowledge1 = And(
    # A can be one and not both a knight and a knave
    Or(AKnight, AKnave),
    Not(And(AKnight, AKnave)),
    # B can be one and not both a knight and a knave
    Or(BKnight, BKnave),
    Not(And(BKnight, BKnave)),
    # test if what A says is true(knight) or false(knave)
    Implication(AKnight, A_says),
    Implication(AKnave, Not(A_says))
)

# Solve the puzzle
puzzle_solver('Puzzle 1', knowledge1)

In [ ]:
# Test knowledge1
test_1(model_check, knowledge1, symbols)

## Puzzle 2
**Puzzle 2** has two characters: A and B.
- A says “We are the same kind.”
- B says “We are of different kinds.”

In [ ]:
# Puzzle 2
# A says "We are the same kind."
A_says = Or(And(AKnight, BKnight), And(AKnave, BKnave))
# B says "We are of different kinds."
B_says = Not(A_says)

knowledge2 = And(
    # A can be one and not both a knight and a knave
    Or(AKnight, AKnave),
    Not(And(AKnight, AKnave)),
    # B can be one and not both a knight and a knave
    Or(BKnight, BKnave),
    Not(And(BKnight, BKnave)),
    # test if what A says is true(knight) or false(knave)
    Implication(AKnight, A_says),
    Implication(AKnave, Not(A_says)),
    # test if what B says is true(knight) or false(knave)
    Implication(BKnight, B_says),
    Implication(BKnave, Not(B_says))
)

# Solve the puzzle
puzzle_solver('Puzzle 2', knowledge2)

In [ ]:
# Test knowledge2
test_2(model_check, knowledge2, symbols)

## Puzzle 3
**Puzzle 3** has three characters: A, B, and C.
- A says either “I am a knight.” or “I am a knave.”, but you don’t know which.
- B says “A said ‘I am a knave.’”
- B then says “C is a knave.”
- C says “A is a knight.”

In [ ]:
# Puzzle 3
# A says either "I am a knight." or "I am a knave.", but you don't know which.
A_says = Or(AKnight, AKnave)
# B says "A said 'I am a knave'." and "C is a knave."
B_says = And(AKnave, CKnave)
# C says "A is a knight."
C_says = AKnight

knowledge3 = And(
    # A can be one and not both a knight and a knave
    Or(AKnight, AKnave),
    Not(And(AKnight, AKnave)),
    # B can be one and not both a knight and a knave
    Or(BKnight, BKnave),
    Not(And(BKnight, BKnave)),
    # C can be one and not both a knight and a knave
    Or(CKnight, CKnave),
    Not(And(CKnight, CKnave)),
    # test if what A says is true(knight) or false(knave)
    Implication(AKnight, A_says),
    Implication(AKnave, Not(A_says)),
    # test if what B says is true(knight) or false(knave)
    Implication(BKnight, B_says),
    Implication(BKnave, Not(B_says)),
    # test if what C says is true(knight) or false(knave)
    Implication(CKnight, C_says),
    Implication(CKnave, Not(C_says))
)

# Solve the puzzle
puzzle_solver('Puzzle 3', knowledge3)

In [ ]:
# Test knowledge3
test_3(model_check, knowledge3, symbols)

## Puzzle 4
**Puzzle 4** has three characters: A, B, and C.
- A says “B never lies.”
- C says “A is a knave and I am a knave.”
- B says “C is a knave.”

In [ ]:
# Puzzle 4
# A says "B never lies."
A_says = BKnight
# C says "A is a knave and I am a knave."
C_says = And(AKnave, CKnave)
# B says "C is a knave."
B_says = CKnave

knowledge4 = And(
    # A can be one and not both a knight and a knave
    Or(AKnight, AKnave),
    Not(And(AKnight, AKnave)),
    # B can be one and not both a knight and a knave
    Or(BKnight, BKnave),
    Not(And(BKnight, BKnave)),
    # C can be one and not both a knight and a knave
    Or(CKnight, CKnave),
    Not(And(CKnight, CKnave)),
    # test if what A says is true(knight) or false(knave)
    Implication(AKnight, A_says),
    Implication(AKnave, Not(A_says)),
    # test if what B says is true(knight) or false(knave)
    Implication(BKnight, B_says),
    Implication(BKnave, Not(B_says)),
    # test if what C says is true(knight) or false(knave)
    Implication(CKnight, C_says),
    Implication(CKnave, Not(C_says))
)

# Solve the puzzle
puzzle_solver('Puzzle 4', knowledge4)

In [ ]:
# Test knowledge4
test_4(model_check, knowledge4, symbols)

## Puzzle 5
**Puzzle 5** has three characters: A, B, and C.
- B says “A is like me”
- A says “C always tells the truth.”
- C says “B lies.”
- B says “A is lying.”

In [ ]:
# Puzzle 5
# A says "C always tells the truth."
A_says = CKnight
# C says "B lies."
C_says = BKnave
# B says "A is my type." and "A is lying."
B_says = And(Not(A_says), Or(And(AKnight, BKnight), And(AKnave, BKnave)))

knowledge5 = And(
    # A can be one and not both a knight and a knave
    Or(AKnight, AKnave),
    Not(And(AKnight, AKnave)),
    # B can be one and not both a knight and a knave
    Or(BKnight, BKnave),
    Not(And(BKnight, BKnave)),
    # C can be one and not both a knight and a knave
    Or(CKnight, CKnave),
    Not(And(CKnight, CKnave)),
    # test if what A says is true(knight) or false(knave)
    Implication(AKnight, A_says),
    Implication(AKnave, Not(A_says)),
    # test if what B says is true(knight) or false(knave)
    Implication(BKnight, B_says),
    Implication(BKnave, Not(B_says)),
    # test if what C says is true(knight) or false(knave)
    Implication(CKnight, C_says),
    Implication(CKnave, Not(C_says))
)

# Solve the puzzle
puzzle_solver('Puzzle 5', knowledge5)

In [ ]:
# Test knowledge5
test_5(model_check, knowledge5, symbols)

# Submission
- Be sure to remove all the `raise NotImplementedError` lines in all the cells above.
- Run all the tests and ensure that there is no error.
- Download your `pset3.ipynb` file and submit it through SOMAS under the Assignment labeled pset2, under Wk 8.